# GKY (2020) Replication: Part 1 - Data Collection & Preparation

##### This notebook handles the first three major steps of the data pipeline:
##### 1.  **Prepare Characteristics Data:** Downloads, cleans, rank-transforms, and imputes the 94 stock-level characteristics from Prof. Xiu's website.
##### 2.  **Prepare CRSP Data:** Fetches stock returns from WRDS, applies GKY filters, and calculates excess returns and lagged market cap.
##### 3.  **Prepare Macro Data:** Downloads and lags the 8 Welch-Goyal macroeconomic predictors.

##### All prepared data is saved to the top-level `/data` directory in the efficient Parquet format.


In [9]:
import pandas as pd
import numpy as np

import ssl
import io
import requests
import zipfile

import os
import gc

import wrds


In [10]:
# Define the sample period as per the GKY paper
START_DATE = '1957-03-01'
END_DATE = '2016-12-31'

# List of the 8 specific macroeconomic predictors required by GKY (2020)
GKY_MACRO_PREDICTORS = [
    'dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy', 'svar'
]


In [11]:
# --- Configuration ---
OUTPUT_DIR = '../data' 
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Define final file paths in one place
CHARACTERISTICS_PATH = os.path.join(OUTPUT_DIR, 'characteristics_prepared.parquet')
CRSP_PATH = os.path.join(OUTPUT_DIR, 'crsp_prepared.parquet')
MACRO_PATH = os.path.join(OUTPUT_DIR, 'macro_predictors_lagged.parquet')


In [12]:
# --- Step 1: Prepare Stock-Level Characteristics ---

def _rank_transform(series: pd.Series) -> pd.Series:
    """
    Internal helper to perform cross-sectional rank transformation on a [-1, 1] interval.
    """
    n_valid = series.notna().sum()
    if n_valid <= 1:
        return series
    ranked_series = series.rank(na_option='keep')
    return 2 * ((ranked_series - 1) / (n_valid - 1) - 0.5)

def prepare_characteristics_data() -> pd.DataFrame:
    """
    Acquires and pre-processes the 94 GKY stock-level characteristics.
    """
    print("\n--- Starting Step 1: Prepare Stock-Level Characteristics ---")
    
    print("Downloading characteristics data from Dacheng Xiu's website...")
    url = "https://dachxiu.chicagobooth.edu/download/datashare.zip"
    response = requests.get(url, verify=False)
    response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open('datashare.csv') as f:
            df = pd.read_csv(f)
    print(f"Successfully downloaded and read {len(df)} rows.")
    
    df.rename(columns={'DATE': 'month'}, inplace=True)
    df['month'] = pd.to_datetime(df['month'], format='%Y%m%d') + pd.offsets.MonthEnd(0)
    
    char_cols = [col for col in df.columns if col not in ['permno', 'month', 'sic2']]
    rename_map = {col: f"characteristic_{col}" for col in char_cols}
    df.rename(columns=rename_map, inplace=True)
    
    df.dropna(subset=['sic2'], inplace=True)
    df['sic2'] = df['sic2'].astype('category')
    
    char_cols_prefixed = list(rename_map.values())
    
    print("Applying cross-sectional rank transformation...")
    df[char_cols_prefixed] = df.groupby('month')[char_cols_prefixed].transform(_rank_transform)
    
    print("Imputing missing values (two-step: median then zero)...")
    df[char_cols_prefixed] = df.groupby('month')[char_cols_prefixed].transform(lambda x: x.fillna(x.median()))
    df[char_cols_prefixed] = df[char_cols_prefixed].fillna(0)
    
    final_df = df[['permno', 'month', 'sic2'] + char_cols_prefixed].reset_index(drop=True)
    
    final_df['permno'] = final_df['permno'].astype('int32')
    for col in char_cols_prefixed:
        final_df[col] = final_df[col].astype('float32')
        
    gc.collect()

    print("--- Characteristics Data Preparation Complete ---")
    return final_df


In [13]:
# --- Step 2: Prepare CRSP Return Data ---

def prepare_crsp_data(db: wrds.Connection) -> pd.DataFrame:
    """
    Fetches and prepares monthly CRSP stock data as per GKY (2020) methodology.
    """
    print("\n--- Starting Step 2: Prepare CRSP Return Data ---")

    print("Fetching CRSP monthly stock data (crsp.msf)...")
    sql_query = f"""
        SELECT 
            a.permno, a.date, a.ret, a.prc, a.shrout,
            b.shrcd, b.exchcd
        FROM 
            crsp.msf AS a
        LEFT JOIN 
            crsp.msenames AS b 
            ON a.permno = b.permno AND b.namedt <= a.date AND a.date <= b.nameendt
        WHERE 
            a.date BETWEEN '{START_DATE}' AND '{END_DATE}'
            AND b.shrcd IN (10, 11)
            AND b.exchcd IN (1, 2, 3)
    """
    crsp_df = db.raw_sql(sql_query, date_cols=['date'])
    
    print("Fetching Fama-French risk-free rate...")
    ff_df = db.raw_sql(
        f"SELECT date, rf FROM ff.factors_monthly WHERE date BETWEEN '{START_DATE}' AND '{END_DATE}'", 
        date_cols=['date']
    )
    
    print("Merging, cleaning, and calculating variables...")
    crsp_df['month'] = crsp_df['date'] + pd.offsets.MonthEnd(0)
    ff_df['month'] = ff_df['date'] + pd.offsets.MonthEnd(0)
    
    df = pd.merge(crsp_df, ff_df[['month', 'rf']], on='month', how='left')
    df['ret'] = pd.to_numeric(df['ret'], errors='coerce')
    df.dropna(subset=['ret', 'rf', 'prc', 'shrout'], inplace=True)

    df['mktcap'] = np.abs(df['prc']) * df['shrout']
    df.sort_values(by=['permno', 'month'], inplace=True)
    df['mktcap_lag'] = df.groupby('permno')['mktcap'].shift(1)
    df['ret_excess'] = df['ret'] - df['rf']
    
    df.dropna(subset=['mktcap_lag', 'ret_excess'], inplace=True)

    final_df = df[['permno', 'month', 'ret_excess', 'mktcap_lag']].reset_index(drop=True)
    final_df = final_df.astype({
        'permno': 'int32', 'ret_excess': 'float32', 'mktcap_lag': 'float64'
    })

    del crsp_df, ff_df, df
    gc.collect()

    print("--- CRSP Data Preparation Complete ---")
    return final_df


In [14]:
# --- Step 3: Prepare Macroeconomic Predictors ---

def prepare_macro_data() -> pd.DataFrame:
    """
    Downloads and prepares the 8 Welch-Goyal macroeconomic predictors.
    """
    print("\n--- Starting Step 3: Prepare Macroeconomic Predictors ---")
    
    sheet_id = "1bM7vCWd3WOt95Sf9qjLPZjoiafgF_8EG"
    sheet_name = "macro_predictors.xlsx"
    macro_link = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    
    print(f"Acquiring macro data from TidyFinance source...")
    response = requests.get(macro_link)
    response.raise_for_status()
    macro_raw = pd.read_csv(io.StringIO(response.text), thousands=",")
        
    macro_processed = (
        macro_raw
        .assign(
            month=lambda x: pd.to_datetime(x["yyyymm"], format="%Y%m") + pd.offsets.MonthEnd(0),
            dp=lambda x: np.log(x["D12"]) - np.log(x["Index"]),
            ep=lambda x: np.log(x["E12"]) - np.log(x["Index"]),
            tms=lambda x: x["lty"] - x["tbl"],
            dfy=lambda x: x["BAA"] - x["AAA"]
        )
        .rename(columns={"b/m": "bm"})
        [['month'] + GKY_MACRO_PREDICTORS]
    )
    
    macro_lagged = macro_processed.copy()
    macro_lagged['month'] = macro_lagged['month'] + pd.DateOffset(months=1)
    
    rename_map = {col: f"macro_{col}" for col in GKY_MACRO_PREDICTORS}
    macro_lagged.rename(columns=rename_map, inplace=True)
    
    del macro_raw, macro_processed
    gc.collect()
    
    print("--- Macro Data Preparation Complete ---")
    return macro_lagged


In [15]:

# --- Main Execution Block ---
if __name__ == "__main__":
    db = None
    try:
        # Step 0: Connect to WRDS
        print("Connecting to WRDS...")
        db = wrds.Connection()
        print("WRDS Connection Successful.")
        
        # Execute data preparation steps
        characteristics_df = prepare_characteristics_data()
        crsp_df = prepare_crsp_data(db)
        macro_df = prepare_macro_data()
        
        # Save the final prepared data
        print(f"\nSaving prepared files to '{OUTPUT_DIR}' directory...")
        characteristics_df.to_parquet(CHARACTERISTICS_PATH, index=False)
        crsp_df.to_parquet(CRSP_PATH, index=False)
        macro_df.to_parquet(MACRO_PATH, index=False)
        print("All files saved successfully.")

    except Exception as e:
        print(f"\nAn error occurred: {e}")
    
    finally:
        # Ensure the connection is always closed
        if db:
            db.close()
            print("\nWRDS Connection Closed.")
            

Connecting to WRDS...
Loading library list...
Done
WRDS Connection Successful.

--- Starting Step 1: Prepare Stock-Level Characteristics ---


/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dachxiu.chicagobooth.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Successfully downloaded and read 4117300 rows.
Applying cross-sectional rank transformation...
Imputing missing values (two-step: median then zero)...
--- Characteristics Data Preparation Complete ---

--- Starting Step 2: Prepare CRSP Return Data ---
Fetching CRSP monthly stock data (crsp.msf)...
Fetching Fama-French risk-free rate...
Merging, cleaning, and calculating variables...
--- CRSP Data Preparation Complete ---

--- Starting Step 3: Prepare Macroeconomic Predictors ---
Acquiring macro data from TidyFinance source...
--- Macro Data Preparation Complete ---

Saving prepared files to '../data' directory...
All files saved successfully.

WRDS Connection Closed.


##### (OPTIONAL) The below is to read and check the stored dataframes:

In [16]:
# # --- Configuration ---
# # Define the directory where your data is stored
# DATA_DIR = '../data' 

# # --- Load the Prepared DataFrames ---
# print("Loading prepared data from parquet files...")

# # Define file paths
# characteristics_path = os.path.join(DATA_DIR, 'characteristics_prepared.parquet')
# crsp_path = os.path.join(DATA_DIR, 'crsp_prepared.parquet')
# macro_path = os.path.join(DATA_DIR, 'macro_predictors_lagged.parquet')

# # Check if files exist before trying to read them
# if not os.path.exists(crsp_path) or not os.path.exists(macro_path):
#     print("Error: One or more parquet files not found.")
#     print("Please run the data preparation script first.")
#     # Exit or handle the error as needed

# # Read the parquet files into pandas DataFrames
# characteristics_df = pd.read_parquet(characteristics_path)
# crsp_df = pd.read_parquet(crsp_path)
# macro_df = pd.read_parquet(macro_path)

# print("Data successfully loaded.")

# # --- Verify the Loaded Data ---
# print("\n--- Characteristics Data Information ---")
# print(characteristics_df.info())
# print("\nHead of Characteristics data:")
# print(characteristics_df.head())

# print("\n--- CRSP Data Information ---")
# print(crsp_df.info())
# print("\nHead of CRSP data:")
# print(crsp_df.head())

# print("\n--- Macro Data Information ---")
# print(macro_df.info())
# print("\nHead of Macro data:")
# print(macro_df.head())
